# API request and csv appender for IQAir data

In [1]:
import pandas as pd
import requests
import json
import csv
from datetime import datetime, timedelta, timezone
from zoneinfo import ZoneInfo
import numpy as np
import matplotlib.dates as mdates
from matplotlib.dates import DateFormatter
import feedparser #May need to add: !pip install feedparser prior to importing
import re

Notes on data:
- Data is directly from air.utah.gov's API, which uses rss feed for SLC
- Website: https://air.utah.gov/airapi.html
- You need a username to use the API.

In [2]:
rss_url = 'https://air.utah.gov/rssCurFeed.php?id=slc'
feed = feedparser.parse(rss_url)

if feed.status == 200:
    print(f'Feed Title: {feed.feed.title}')
    
# Iterate through the entries (articles) in the feed
    for entry in feed.entries:
        print(f'\nTitle: {entry.title}')
        print(f'Link: {entry.link}')
        print(f'Published {entry.published}')

else:
    print(f'Failed to retrieve RSS feed. Status code: {feed.status}')

Feed Title: Utah DEQ: DAQ: Current Conditions RSS Feed: slc

Title: Salt Lake Current Conditions for 11/07/2025 13:00
Link: https://air.utah.gov/currentconditions.php?id=slc
Published Fri, 07 Nov 2025 15:02:51 -0700


In [3]:
if feed.status == 200:
    print(f'Description: {entry.description}')

Description: Ozone: 0.031 ppm

					PM 2.5: 7.9 µg/m3

					Temperature: 56° F

					Wind: S 2 mph


In [4]:
match = re.search(r'PM\s*2\.5:\s*([\d.]+)', entry.description)
if match:
    PM25value = float(match.group(1))
    print(PM25value)

7.9


In [5]:
match = re.search(r'(\d{2}/\d{2}/\d{4} \d{2}:\d{2})', entry.title)
if match:
    datestr = match.group(1)
datestr

'11/07/2025 13:00'

In [6]:
dt = datetime.strptime(datestr, '%m/%d/%Y %H:%M')
# Define MST timezone (UTC-7 for standard time — no DST)
MST = timezone(timedelta(hours=-7))

# Convert to MT (auto DST handling)
dt_mt = dt.replace(tzinfo=ZoneInfo("America/Denver"))
print(f'MT: {dt_mt}')

# Convert to MST
dt_mst = dt_mt.astimezone(MST)
dt_mst_naive = dt_mst.replace(tzinfo=None)
print(f'MST: {dt_mst_naive}')

formatted = dt_mst.strftime('%Y-%m-%d %H:%M:%S')
print(f'As a string: {formatted}')

MT: 2025-10-18 00:00:00-06:00
MST: 2025-10-17 23:00:00
As a string: 2025-10-17 23:00:00


In [7]:
headers = ['Timestamp','PM2.5 (ug/m3)']
headers

['Timestamp', 'PM2.5 (ug/m3)']

In [8]:
df = pd.DataFrame([formatted, PM25value]).T
df

,0,1
0,2025-10-17 23:00:00,4.1


In [9]:
df.columns = headers
df

,Timestamp,PM2.5 (ug/m3)
0,2025-10-17 23:00:00,4.1


In [10]:
new_data = df.iloc[0].tolist()
new_data

['2025-10-17 23:00:00', 4.1]

In [11]:
def append_to_csv(file_path, data_row):
    """
    Appends a single row of data to an existing CSV file.

    Args:
        file_path (str): The path to the CSV file.
        data_row (list): A list representing the row of data to append.
    """
    try:
        with open(file_path, 'a', newline='') as csvfile:
            writer = csv.writer(csvfile)
            writer.writerow(data_row)
        print(f"Data '{data_row}' appended successfully to '{file_path}'.")
    except FileNotFoundError:
        print(f"Error: The file '{file_path}' was not found.")
    except Exception as e:
        print(f"An error occurred: {e}")

In [15]:
csv_file = 'AirUtah_API_dump.csv'

#Check that this data call is new data by checking last timestamp
df_existingdata = pd.read_csv(csv_file)
print(df_existingdata.head())

             Timestamp  PM2.5 (ug/m3)
0  2025-10-17 21:00:00            4.5


In [16]:
print(df_existingdata.tail(1))

             Timestamp  PM2.5 (ug/m3)
0  2025-10-17 21:00:00            4.5


In [17]:
last_timestamp = df_existingdata.iloc[-1,0]
print(last_timestamp)

2025-10-17 21:00:00


In [18]:
#Turn last timestamp into datetime object
format_string = "%Y-%m-%d %H:%M:%S"
last_timestamp_object = datetime.strptime(last_timestamp, format_string)
print(last_timestamp_object)

2025-10-17 21:00:00


In [19]:
#If this hour's data is not yet appended to the csv file, append data.
if last_timestamp_object < dt_mst_naive:
    append_to_csv(csv_file,new_data)
else:
    print('last_timestamp_object is not less than current timestamp')

Data '['2025-10-17 23:00:00', 4.1]' appended successfully to 'AirUtah_API_dump.csv'.
